## <center>Мини-проект. Полный анализ данных и построение прогнозной модели модели 
<center> Автор: Григорий Демин

В рамках данного задания используется набор данных по директ-маркетингу португальского банка. Ссылка: [Bank Marketing Data Set](http://archive.ics.uci.edu/ml/datasets/Bank+Marketing). 

In [ ]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
%pylab inline
import seaborn as sns
import pandas as pd

__Описание набора данных__

Данные описывают директ-маркетинговую кампанию португальского банка. Кампания заключалась в обзвоне клиентов и предложения им депозита. Достаточно часто одному и тому же клиенту делалось несколько звонков.
Целью задачи является предсказание, воспользуется ли клиент депозитом или нет.


__Список переменных:__
* 1 - Возраст (шкала)
* 2 - job : Род занятий (номинальная: "admin." (админ. персонал),"unknown" (неизвестно),"unemployed" (безработный),"management" (менеджмент),"housemaid" (домохозяйка),"entrepreneur" (предприниматель),"student" (учащийся),"blue-collar" (служащий),"self-employed" (самозанятый),"retired" (пенсионер),"technician","services" (сервис) 
* 3 - marital : Семейное положение (номинальная: "married" (женат/замужем),"divorced" (разведен(а)/вдова/вдовец),"single" (одинокий))
* 4 - education: Образование (номинальная: "unknown" (неизвестно),"secondary" (среднее),"primary" (начальное),"tertiary" (высшее))
* 5 - default: была ли просрочка по кредиту? (бинарная: "yes" (да),"no" (нет))
* 6 - balance: среднегодовой баланс на счету, в евро (число) 
* 7 - housing: есть ли ипотека? (бинарная: "yes" (да),"no" (нет))
* 8 - loan: есть ли личные кредиты? (бинарная: "yes" (да),"no" (нет))
* _далее переменные, связанные с предыдущими контактами с данным клиентом:_
* 9 - contact: тип коммуникации (номинальная: "unknown" (неизвестно),"telephone" (стационарный телефон),"cellular"(сотовый)) 
* 10 - day: число месяца, когда был последний контакт (число)
* 11 - month: месяц, когда был последний контакт (Номинальная: "jan", "feb", "mar", ..., "nov", "dec")
* 12 - duration: длительность последнего контакта, в секундах (число)
* _другие атрибуты:_
* 13 - campaign: количество контактов, которое было с данным клиентом в данной кампании (число, включает последний контакт)
* 14 - pdays: количество дней, которое прошло с последнего контакта данной кампании (число, -1 обозначает, что ранее контактов не было)
* 15 - previous: количество контактов, которое было с данным клиентом до этой кампании (число)
* 16 - poutcome: результат предыдущей кампании (номинальная: "unknown" (неизвестно),"other" (другое),"failure" (неудача),"success" (успех))

  Целевая переменная:
  17 - y: Открыл ли клиент депозит? (бинарная: 1, 0)

In [ ]:
data = pd.read_csv("../../data/bank.csv")

In [ ]:
data.head() 

**Выводим основные харакетристики переменных;**

In [ ]:
print(data.shape)
data.describe(include = "all").T

In [ ]:
# частотки по категориальным переменным
categorical = ["marital","education","default","housing","loan","contact","month","poutcome"]
for each_var in categorical:
    print('********')
    print('*',each_var,'*')
    print(data[each_var].value_counts())


Делаем визуализацию:

In [ ]:
sns.pairplot(data)

Далее предобработка данных. 
* Удалим переменную duration - это длительность последнего контакта, если она равна нулю, то явно, что успеха не было. Из-за этого у неё хорошая прогнозная сила, но её значение не известно до звонка.
* pdays - количество дней, прошедшее с последнего звонка. Если -1, то звонка не было. Соответсвенно, перекодируем -1 в отдельную переменную - "Был звонок/не был", а в исходной переменной -1 заменим на медиану.
* В дополнение к непрерывному возрасту сделаем еще возраст категориальный.
* Перекодируем все категориальные переменные в dummies (0,1).
* Масштабируем все переменные.
* Разобъем выборку на тестовую и обучающую.

In [ ]:
### Обрабатываем pdays:
first_call = (data.pdays == -1).astype(int)
first_call.name = "first_call"
data.pdays[data.pdays==-1] = NaN
data.pdays = data.pdays.fillna(value = data.pdays.median())
### Делаем возрастные категории
age1 = (data.age<25).astype(int)
age2 = (data.age>50).astype(int)
age1.name = "age1"
age2.name = "age2"


In [ ]:
### dummy переменные:
data_dummies = pd.concat([
                        pd.get_dummies(data.job     , prefix = 'job'),
                        pd.get_dummies(data.marital , prefix = 'marital'),
                        pd.get_dummies(data.education, prefix = 'education'),
                        pd.get_dummies(data.default , prefix = 'default'),
                        pd.get_dummies(data.housing , prefix = 'housing'),
                        pd.get_dummies(data.loan , prefix = 'loan'),
                        pd.get_dummies(data.contact , prefix = 'contact'),
                        pd.get_dummies(data.month   , prefix = 'month'),
                        pd.get_dummies(data.poutcome, prefix = 'poutcome'),
                        data.age            ,
                        data.balance        ,
                        data.day            ,
                        data.campaign       ,
                        data.pdays          ,
                        data.previous       ,
                        first_call,
                        age1,
                        age2], axis=1)

Разбиваем на тестовую и обучающую выборку

In [ ]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_dummies, data.y, 
                                                    test_size=0.3, 
                                                    random_state=20160212,
                                                   stratify = data.y)

Масштабируем переменные

In [ ]:
from sklearn import preprocessing
X_train = preprocessing.scale(X_train)
X_test = preprocessing.scale(X_test)
### Конвертируем назад в Pandas DataFrame
X_train = pd.DataFrame(X_train)
X_train.columns = data_dummies.columns
X_test = pd.DataFrame(X_test)
X_test.columns = data_dummies.columns

**Посмотрим на baseline - классификацию случайным лесом без настройки параметров.**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
forest = RandomForestClassifier(n_estimators=500)
forest.fit(X_train, y_train)
test_pred = forest.predict(X_test)

Убедимся в том, что для данной задачи доля правильных ответов не будет хорошей метрикой качества модели.

In [ ]:
accuracy_score(y_test, test_pred)

Однако, лучшее качество дает даже тривиальный прогноз, что клиент не откроет депозит.

In [ ]:
y_test.value_counts()[0] / y_test.shape[0]

In [ ]:
f1_score(y_test, test_pred)

In [ ]:
confusion_matrix(y_test, test_pred)

**Выберем наиболее важные переменные**

In [ ]:
forest = RandomForestClassifier(n_estimators=1000, max_depth = 5,
                                random_state=42).fit(X_train, y_train)

features = pd.DataFrame(forest.feature_importances_,
                        index=X_train.columns, 
                        columns=['Importance']).sort(['Importance'], 
                                                     ascending=False)
features

In [ ]:
plt.plot(range(len(features.Importance.tolist())), 
         features.Importance.tolist())

Выбираем 20 признаков. Конвертируем выборки в Numpy матрицы.

In [ ]:
selected_attr = features.index.tolist()[0:20]
X_train = X_train[selected_attr].as_matrix()
X_test = X_test[selected_attr].as_matrix()

Попробуем четыре разных классификатора: логистическую регрессию, Gradient boosting, Random Forest и SVM. Так как у нас сильный дисбаланс в выборке (успехов только 10%), то в качестве меры будем использовать F1 score.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import f1_score

classifiers = [LogisticRegression(),
               GradientBoostingClassifier(), 
               RandomForestClassifier(), 
               SVC()]
classifiers_name = ["LogisticRegression",
               "GradientBoostingClassifier", 
               "RandomForestClassifier", 
               "SVC"]


Настраиваем параметры выбранных алгоритмов с помощью GridSearchCV и выбираем лучший классификатор.

In [ ]:
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold

n_folds = 5
scores = []
fits = []
logistic_params = {'penalty': ('l1', 'l2'), 
               'C': (.01,.1,1,5)}
gbm_params = { 'n_estimators': [100, 300, 500],
               'learning_rate':(0.1, 0.5, 1),
                'max_depth': list(range(3, 6)), 
               'min_samples_leaf': list(range(10, 31, 10))}
forest_params = {'n_estimators': [100, 300, 500],
                'criterion': ('gini', 'entropy'), 
               'max_depth': list(range(3, 6)), 
               'min_samples_leaf': list(range(10, 31, 10))}

svm_param = {'kernel' : ('linear', 'rbf'),
              'C': (.5, 1, 2)           
             }
params = [logistic_params, gbm_params, forest_params, svm_param]

# Проводим кросс-валидацию для всех моделей
for i, each_classifier in enumerate(classifiers):
    clf = each_classifier
    clf_params = params[i]
    grid = GridSearchCV(clf, clf_params, 
                        cv=StratifiedKFold(y_train, n_folds=n_folds,
                        shuffle=False, random_state=42), 
                        n_jobs=-1, scoring="f1")
    grid.fit(X_train, y_train)
    fits.append(grid.best_params_)
    clf_best_score = grid.best_score_
    scores.append(clf_best_score)
    print(classifiers_name[i], clf_best_score, "\n", grid.best_params_)

In [ ]:
# Печатаем параметры лучшего
grid_value = max(scores)
grid_index = [i for i in xrange(len(scores)) if scores[i]==grid_value][0]
print("Лучший классификатор при GridSearch:",
      classifiers_name[grid_index], grid_value)
print(fits[grid_index])

**Для лучшего классификатора и набора параметров для него выберем более мелкую сетку (для того чтобы уточнить лучшие значения параметров).**

In [ ]:
clf_params = {'n_estimators': (250, 300, 350), 
              'learning_rate': (0.75, 1, 1.25, 1.5), 
              'min_samples_leaf': list(range(1, 14, 3))}

clf = classifiers[grid_index]
grid = GridSearchCV(clf, clf_params, cv=n_folds, 
                    n_jobs=-1, scoring="f1")
grid.fit(X_train, y_train)
clf_best_score = grid.best_score_
clf_best_params = grid.best_params_
clf_best = grid.best_estimator_
mean_validation_scores = []
print("Лучший результат", clf_best_score, 
      "лучшие параметры", clf_best_params)

**Строим график кривой обучения**

In [ ]:
def plot_with_std(x, data, **kwargs):
        mu, std = data.mean(1), data.std(1)
        lines = plt.plot(x, mu, '-', **kwargs)
        plt.fill_between(x, mu - std, mu + std, edgecolor='none',
                         facecolor=lines[0].get_color(), alpha=0.2)
        
def plot_learning_curve(clf, X, y, scoring, cv=5):
 
    train_sizes = np.linspace(0.05, 1, 20)
    n_train, val_train, val_test = learning_curve(clf,
                                                  X, y, train_sizes, cv=cv,
                                                  scoring=scoring)
    plot_with_std(n_train, val_train, label='training scores', c='green')
    plot_with_std(n_train, val_test, label='validation scores', c='red')
    plt.xlabel('Training Set Size'); plt.ylabel(scoring)
    plt.legend()

In [ ]:
plot_learning_curve(GradientBoostingClassifier(n_estimators=2, 
                    learning_rate=1.5, min_samples_leaf=7),
                   X_train, y_train, scoring='f1', cv=10)

**Построим валидационную кривую для данных параметров бустинга. В качестве параметра сложности будем использовать learning_rate:**

In [ ]:
from sklearn.learning_curve import validation_curve

def plot_validation_curve(clf, X, y, cv_param_name, 
                          cv_param_values, scoring):

    val_train, val_test = validation_curve(clf, X, y, cv_param_name,
                                           cv_param_values, cv=5,
                                                  scoring=scoring)
    plot_with_std(cv_param_values, val_train, 
                  label='training scores', c='green')
    plot_with_std(cv_param_values, val_test, 
                  label='validation scores', c='red')
    plt.xlabel(cv_param_name); plt.ylabel(scoring)
    plt.legend()

In [ ]:
learning_rates = np.linspace(0.1, 2.3, 20)
plot_validation_curve(GradientBoostingClassifier(n_estimators=250, 
                    min_samples_leaf=7), X_train, y_train, 
                      cv_param_name='learning_rate', 
                      cv_param_values=learning_rates,
                   scoring='f1')

In [ ]:
final_gbm = GradientBoostingClassifier(n_estimators=250, 
                    min_samples_leaf=7, learning_rate=1.5)
final_gbm.fit(X_train, y_train)
final_pred = final_gbm.predict(X_test)
accuracy_score(y_test, final_pred), f1_score(y_test, final_pred)

__Выводы.__

Построена модель предсказания, согласится ли респондент на банковский продукт. Модель предсказывает с 87%-ной долей правильных ответов на отложенных 30% выборки. Но accuracy не очень хорошо характеризует качество модели из-за сильного дисбаланса в целевой переменной (~90% против ~10%), поэтому в качестве целевой была выбрана метрика F1-score. На отложенной выборке удалось добиться только F1=0.22.
Построены кривые обучения и валидационные кривые. К сожалению, у них наблюдается некоторая немонотонность, однако видно, что увеличение количества примеров более 2000 не приносит существенной выгоды (у нас в обучающей выборке более 3000 примеров). Из валидационного графика видно, что learning rate = 1.5, выбранный с помощью перебора на обучающей выборки, близок к оптимальному.  